<a href="https://colab.research.google.com/github/emanbuc/LLM-playground/blob/master/TestAutomazioneMassime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test Estrazione Massima

In [ ]:
!pip install OpenAI bs4

In [ ]:
!pip freeze > requirements.txt

Per caricare API KEY da variabili di ambiente



```
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']
```



In [ ]:
#carica API KEY da secret di Google Colab
from google.colab import userdata
openai_key= userdata.get('openai-key-nomos-ai')

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=openai_key)

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
def get_completion_and_token_count(messages,
                                   model="gpt-3.5-turbo", #model="gpt-4-turbo-preview", #model="gpt-3.5-turbo",
                                   temperature=0,
                                   max_tokens=500):

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    content = response.choices[0].message.content

    token_dict = {
'prompt_tokens':response.usage.prompt_tokens,
'completion_tokens':response.usage.completion_tokens,
'total_tokens':response.usage.total_tokens,
    }

    return content, token_dict

In [ ]:
config= {
    "massime_path": "/gdrive/MyDrive/NomosAI/Massime/test09",#"/gdrive/MyDrive/NomosAI/Massime",
    "sentenze_path": "/gdrive/MyDrive/NomosAI/Docs-Condivisi-NomosAI/Test Gold Standard", #"/gdrive/MyDrive/NomosAI/Sentenze",
    "prompt_path":"/gdrive/MyDrive/NomosAI/Prompt",
    "models":["gpt-4o-2024-05-13","gpt-4-turbo-2024-04-09","gpt-3.5-turbo"],
    "prompts":["system_message_E.txt"],#["system_message_A.txt","system_message_B.txt","system_message_C.txt"],
    "debug": False
}

In [51]:
config["massime_path"]

'/gdrive/MyDrive/NomosAI/Massime/test09'

In [ ]:
def create_text_file_dict_from_folder(folder_path):
    import os
    import glob
    # Initialize an empty dictionary
    text_files_dict = {}

    # Create the pattern for .txt files
    pattern = os.path.join(folder_path, '*.txt')

    # Find all .txt files matching the pattern
    for file_path in glob.glob(pattern):
        # Extract the file name from the file path
        file_name = os.path.basename(file_path)

        # Read the content of the file
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

        # Add the file name and content to the dictionary
        text_files_dict[file_name] = content

    return text_files_dict

In [ ]:
prompts_files_dict = create_text_file_dict_from_folder(config["prompt_path"])
for file_name, content in prompts_files_dict.items():
    print(f'File: {file_name}\n')

In [ ]:
def save_output_as_text_file(filename,text_content):
  percorso_file = os.path.join(config["massime_path"], filename)
  with open(percorso_file+'.txt', 'w') as f:
    f.write(text_content)

In [ ]:
def genera_massima(prompt_file,sentenza,model,output_file_name):

    prompt = prompts_files_dict[prompt_file]
    messages = [
      {'role':'system',
      'content':prompt},
      {'role':'user',
      'content':sentenza},
      ]

    if(config["debug"]):
      print("====== promt: ========== \n")
      print(prompt)

      print(" ======== sentenza =======\n")
      print(sentenza)


    response, token_dict = get_completion_and_token_count(messages,model=model,max_tokens= 1000)

    if(config["debug"]):
      print(" ======= response =========\n")

      print(response)

      print("======== tokens_dict ========= \n")

      print(token_dict)
    save_output_as_text_file(output_file_name+"_"+model+"_"+prompt_file,response)

In [ ]:
from bs4 import BeautifulSoup

import os
for radice, cartelle, file in os.walk(config["sentenze_path"]):
    for f in file:
      if f.endswith('.html'):
        percorso_file = os.path.join(radice, f)
        with open(percorso_file, 'r', encoding='utf-8') as fh:
          sentenza = fh.read()

          print("Elaboro file: "+ percorso_file + "\n")

          soup = BeautifulSoup(sentenza, 'html.parser')

          # Estrai il testo da tutti i tag <p>
          testoSentenza = ''
          for paragrafo in soup.find_all('p'):
              testoSentenza += paragrafo.text + '\n'

          save_output_as_text_file(f+"_TESTO_SENTENZA",testoSentenza)

          for prompt_file in config["prompts"]:
            for model in config["models"]:
              try:
                genera_massima(prompt_file,testoSentenza,model,f)
              except:
                print("ERRORE sentenza: "+ percorso_file + " MODEL: "+model+"+ PROMPT: "+ prompt_file +"\n")
